## 핵심 아이디어
1. 메뉴명을 명사 단위로 토큰화
2. 토큰화한 메뉴명을 빈도분석
3. 빈도 내림차순으로 나열한 뒤 특정 비율의 토큰을 채택
4. 채택한 토큰으로 끝나는 메뉴명을 토큰 카테고리로 분류

## 간단하지만 강력한 알고리즘
대부분 한국어 메뉴명이 분류로 종결된다는 특성을 이용하였습니다  
ex)오곡라떼 => '라떼'라는 분류명으로 종결됨

따라서 이렇게 1차 분류를 시도해보고, 오분류되었거나 미분류된 음식명을 2차 분류해보는건 어떠할지?  

In [4]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
# 데이터 임포팅 
df = pd.read_csv('menu_concated_v1.3')
df.dropna(inplace=True)
menu_list = df['menu'].tolist()

In [48]:
# 명사 단위 토큰화
from konlpy.tag import Okt
from tqdm import tqdm

t = Okt()
menu_list = [t.nouns(word) for word in tqdm(menu_list)]

100%|██████████| 192332/192332 [01:47<00:00, 1793.71it/s]


In [53]:
# 토큰화된 리스트의 리스트 내 리스트 제거 
menu_list = [v for word in menu_list for v in word]
menu_list[:10]

['정식', '짜장면', '짬뽕', '우동', '울면', '간짜장', '사천짜장', '삼선짜장', '유니짜장', '삼선']

In [54]:
# 토큰 빈도분석 
count_dict = {}
for i in tqdm(menu_list):
  if i not in count_dict:
    count_dict[i] = 1
  else:
    count_dict[i] += 1


100%|██████████| 365109/365109 [00:00<00:00, 1703081.07it/s]


In [82]:
# 토큰을 빈도 내림차순으로 정렬
items = count_dict.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True) 

# 빈도 상위 10퍼센트의 토큰만 선택
cut_line = int(len(sorted_items) / 10)

# 단, 두글자 이상인 토큰만 선택 (1글자 토큰은 결과를 곱창내놓았음;; 하지만 불용어 설정을 잘 한다면??)
outliner_vocab = [k for k, v in sorted_items[:cut_line] if len(k)!=1]
outliner_vocab[:10]

['라떼', '치즈', '세트', '치킨', '추가', '맥주', '소주', '튀김', '새우', '볶음']

In [83]:
# 전체 메뉴명을 중복 없이 리스트로 가져옴 
menu = df['menu'].tolist()
menu = list(set(menu))
len(menu)

51437

In [ ]:
#토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류!!
menu_cluster = {}

for food_token in outliner_vocab:
  for food_name in menu:
    if food_name not in menu_cluster: # 이미 분류된 메뉴는 건들지 않음

      # 메뉴명이 토큰명보다 길면서 동시에 토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류 
      if len(food_token) <= len(food_name) and food_name[-len(food_token):] == food_token:
        menu_cluster[food_name] = food_token

In [86]:
# 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
df_menu['category'].unique()

array(['라떼', '치즈', '세트', '치킨', '추가', '맥주', '소주', '튀김', '새우', '볶음', '이드',
       '아메리카노', '피자', '구이', '스무디', '볶음밥', '불고기', '초코', '사리', '요거트', '음료수',
       '크림', '바닐라', '자몽', '레몬', '밀크', '해물', '우동', '공기밥', '딸기', '전골',
       '돈까스', '카라멜', '과일', '짬뽕', '라면', '정식', '샐러드', '만두', '한우', '김밥',
       '탕수육', '양념', '고구마', '오징어', '족발', '김치', '삼겹살', '감자', '갈비', '카페모카',
       '메뉴', '된장찌개', '떡볶이', '녹차', '막걸리', '카푸치노', '주스', '소고기', '망고',
       '김치찌개', '돼지', '보쌈', '수육', '에스프레소', '스테이크', '파스타', '커피', '브루',
       '아이스티', '비빔밥', '낙지', '칼국수', '전복', '블랙', '콜라', '냉면', '마끼아또', '순대',
       '음료', '블루베리', '야채', '두부', '허니', '마늘', '오리', '참치', '간장', '민트', '등심',
       '오뎅', '허브', '복분자', '고기', '제육', '베이컨', '얼그레이', '닭발', '스페셜', '버섯',
       '매운탕', '고추', '초밥', '우럭', '코스', '광어', '사이다', '그린티', '바나나', '쥬스',
       '찌개', '오렌지', '연어', '조림', '국밥', '국수', '철판', '모카', '동태', '백세주',
       '샌드위치', '갈릭', '화이트', '토마토', '유자', '아이스', '삼선', '계란', '매콤', '마리',
       '홍차', '잡채', '삼겹', '어묵', '물회', '해장국', '꼬치', '헤이즐넛', '생선', '버거',
 

In [96]:
pd.set_option('display.max_row', 100)

In [97]:
df_menu.sample(100)

,title,category
1068,콜비잭황치즈,치즈
27490,원주풍천장어,장어
14855,모카카푸치노,카푸치노
25112,돈까스도시락,도시락
15048,블루베리바나나주스,주스
10084,캐서롤공기밥,공기밥
13902,오소맛감자,감자
19200,김치고등어찌개,찌개
10999,탕수육짬뽕,짬뽕
23584,막썰어문어,문어
